## Preprocessing

In [101]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from keras.callbacks import ModelCheckpoint
#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [102]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
dropped_columns = ['EIN', 'NAME']
application_df = application_df.drop(columns = dropped_columns)

In [103]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [104]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [105]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff_high = 600
cutoff_low = 20
application_types_to_replace = []
for item in application_df['APPLICATION_TYPE'].value_counts().items():
    if item[1] < cutoff_high and item[1] > cutoff_low:
        application_types_to_replace.append(item[0])
    elif item[1] < cutoff_low:
        application_types_to_replace.append(item[0])
        

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: count, dtype: int64

In [106]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [107]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df['CLASSIFICATION'].value_counts().loc[lambda x : x>1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [108]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_high = 500
cutoff_low = 2
classifications_to_replace = []
for item in application_df['CLASSIFICATION'].value_counts().items():
    if item[1] < cutoff_high and item[1] > cutoff_low:
        classifications_to_replace.append(item[0])
    elif item[1] < cutoff_low:
        classifications_to_replace.append(item[0])
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1474
C7000      777
C1267        2
C1256        2
C1234        2
C1246        2
C3200        2
Name: count, dtype: int64

In [109]:
# Convert categorical data to numeric with `pd.get_dummies`
converted_data = pd.get_dummies(data=application_df, columns=list(set(['APPLICATION_TYPE', 'CLASSIFICATION', 'AFFILIATION',
                                                             'USE_CASE', 'ORGANIZATION', 'SPECIAL_CONSIDERATIONS',
                                                             'INCOME_AMT'])-set(dropped_columns)))

In [110]:
# Split our preprocessed data into our features and target arrays
y = converted_data['IS_SUCCESSFUL']
X = converted_data.drop(columns = 'IS_SUCCESSFUL')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.25)

In [111]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [118]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn = Sequential()

# First hidden layer
nn.add(Dense(units=90, activation = 'relu', input_dim=X_test.shape[1]))

# Second hidden layer
nn.add(Dense(units=45, activation='sigmoid'))

## Third hidden layer??
nn.add(Dense(units = 20, activation = 'sigmoid'))

## Fourth hidden layer
nn.add(Dense(units = 10, activation = 'relu'))

# Output layer
nn.add(Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_41 (Dense)            (None, 90)                4410      
                                                                 
 dense_42 (Dense)            (None, 45)                4095      
                                                                 
 dense_43 (Dense)            (None, 20)                920       
                                                                 
 dense_44 (Dense)            (None, 10)                210       
                                                                 
 dense_45 (Dense)            (None, 1)                 11        
                                                                 
Total params: 9646 (37.68 KB)
Trainable params: 9646 (37.68 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [119]:
# Compile the model
nn.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [120]:
#Checkpoint
checkpoint = ModelCheckpoint("optimized_5.h5", monitor='accuracy', verbose=1,
    save_best_only=True, mode='auto', save_freq=5)

In [121]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=40, callbacks=[checkpoint])

Epoch 1/40
  1/804 [..............................] - ETA: 7:45 - loss: 0.6845 - accuracy: 0.5938
Epoch 1: accuracy improved from -inf to 0.64375, saving model to optimized_5.h5
  5/804 [..............................] - ETA: 11s - loss: 0.6666 - accuracy: 0.6438 
Epoch 1: accuracy did not improve from 0.64375

Epoch 1: accuracy did not improve from 0.64375

Epoch 1: accuracy did not improve from 0.64375

Epoch 1: accuracy did not improve from 0.64375

Epoch 1: accuracy did not improve from 0.64375

Epoch 1: accuracy did not improve from 0.64375

Epoch 1: accuracy did not improve from 0.64375
 41/804 [>.............................] - ETA: 2s - loss: 0.6791 - accuracy: 0.5998 
Epoch 1: accuracy did not improve from 0.64375

Epoch 1: accuracy did not improve from 0.64375

Epoch 1: accuracy did not improve from 0.64375

Epoch 1: accuracy did not improve from 0.64375

Epoch 1: accuracy improved from 0.64375 to 0.64808, saving model to optimized_5.h5
 65/804 [=>............................

430/804 [===============>..............] - ETA: 1s - loss: 0.5957 - accuracy: 0.7107
Epoch 1: accuracy improved from 0.71068 to 0.71092, saving model to optimized_5.h5

Epoch 1: accuracy improved from 0.71092 to 0.71143, saving model to optimized_5.h5
440/804 [===============>..............] - ETA: 1s - loss: 0.5951 - accuracy: 0.7114
Epoch 1: accuracy did not improve from 0.71143

Epoch 1: accuracy did not improve from 0.71143

Epoch 1: accuracy did not improve from 0.71143

Epoch 1: accuracy did not improve from 0.71143

Epoch 1: accuracy did not improve from 0.71143

Epoch 1: accuracy did not improve from 0.71143

Epoch 1: accuracy improved from 0.71143 to 0.71158, saving model to optimized_5.h5
475/804 [================>.............] - ETA: 1s - loss: 0.5938 - accuracy: 0.7116
Epoch 1: accuracy improved from 0.71158 to 0.71159, saving model to optimized_5.h5

Epoch 1: accuracy improved from 0.71159 to 0.71198, saving model to optimized_5.h5
485/804 [=================>............]

133/804 [===>..........................] - ETA: 1s - loss: 0.5608 - accuracy: 0.7263
Epoch 2: accuracy did not improve from 0.74148

Epoch 2: accuracy did not improve from 0.74148

Epoch 2: accuracy did not improve from 0.74148

Epoch 2: accuracy did not improve from 0.74148

Epoch 2: accuracy did not improve from 0.74148

Epoch 2: accuracy did not improve from 0.74148

Epoch 2: accuracy did not improve from 0.74148

Epoch 2: accuracy did not improve from 0.74148
172/804 [=====>........................] - ETA: 0s - loss: 0.5636 - accuracy: 0.7235
Epoch 2: accuracy did not improve from 0.74148

Epoch 2: accuracy did not improve from 0.74148

Epoch 2: accuracy did not improve from 0.74148

Epoch 2: accuracy did not improve from 0.74148

Epoch 2: accuracy did not improve from 0.74148

Epoch 2: accuracy did not improve from 0.74148

Epoch 2: accuracy did not improve from 0.74148

Epoch 2: accuracy did not improve from 0.74148
213/804 [======>.......................] - ETA: 0s - loss: 0.564


Epoch 3: accuracy did not improve from 0.74148

Epoch 3: accuracy did not improve from 0.74148

Epoch 3: accuracy did not improve from 0.74148

Epoch 3: accuracy did not improve from 0.74148
 39/804 [>.............................] - ETA: 1s - loss: 0.5702 - accuracy: 0.7019
Epoch 3: accuracy did not improve from 0.74148

Epoch 3: accuracy did not improve from 0.74148

Epoch 3: accuracy did not improve from 0.74148

Epoch 3: accuracy did not improve from 0.74148

Epoch 3: accuracy did not improve from 0.74148

Epoch 3: accuracy did not improve from 0.74148

Epoch 3: accuracy did not improve from 0.74148

Epoch 3: accuracy did not improve from 0.74148
 77/804 [=>............................] - ETA: 0s - loss: 0.5549 - accuracy: 0.7196
Epoch 3: accuracy did not improve from 0.74148

Epoch 3: accuracy did not improve from 0.74148

Epoch 3: accuracy did not improve from 0.74148

Epoch 3: accuracy did not improve from 0.74148

Epoch 3: accuracy did not improve from 0.74148

Epoch 3: accura

726/804 [==========================>...] - ETA: 0s - loss: 0.5553 - accuracy: 0.7271
Epoch 3: accuracy did not improve from 0.74148

Epoch 3: accuracy did not improve from 0.74148

Epoch 3: accuracy did not improve from 0.74148

Epoch 3: accuracy did not improve from 0.74148

Epoch 3: accuracy did not improve from 0.74148

Epoch 3: accuracy did not improve from 0.74148

Epoch 3: accuracy did not improve from 0.74148

Epoch 3: accuracy did not improve from 0.74148
764/804 [===========================>..] - ETA: 0s - loss: 0.5551 - accuracy: 0.7276
Epoch 3: accuracy did not improve from 0.74148

Epoch 3: accuracy did not improve from 0.74148

Epoch 3: accuracy did not improve from 0.74148

Epoch 3: accuracy did not improve from 0.74148

Epoch 3: accuracy did not improve from 0.74148

Epoch 3: accuracy did not improve from 0.74148

Epoch 3: accuracy did not improve from 0.74148

Epoch 3: accuracy did not improve from 0.74148
804/804 [==============================] - 1s 1ms/step - loss: 0

...


Epoch 37: accuracy did not improve from 0.81250

Epoch 37: accuracy did not improve from 0.81250

Epoch 37: accuracy did not improve from 0.81250

Epoch 37: accuracy did not improve from 0.81250

Epoch 37: accuracy did not improve from 0.81250

Epoch 37: accuracy did not improve from 0.81250

Epoch 37: accuracy did not improve from 0.81250
 43/804 [>.............................] - ETA: 0s - loss: 0.5391 - accuracy: 0.7362
Epoch 37: accuracy did not improve from 0.81250

Epoch 37: accuracy did not improve from 0.81250

Epoch 37: accuracy did not improve from 0.81250

Epoch 37: accuracy did not improve from 0.81250

Epoch 37: accuracy did not improve from 0.81250

Epoch 37: accuracy did not improve from 0.81250

Epoch 37: accuracy did not improve from 0.81250

Epoch 37: accuracy did not improve from 0.81250

Epoch 37: accuracy did not improve from 0.81250
 86/804 [==>...........................] - ETA: 0s - loss: 0.5428 - accuracy: 0.7355
Epoch 37: accuracy did not improve from 0.81250


Epoch 37: accuracy did not improve from 0.81250

Epoch 37: accuracy did not improve from 0.81250

Epoch 37: accuracy did not improve from 0.81250

Epoch 37: accuracy did not improve from 0.81250

Epoch 37: accuracy did not improve from 0.81250

Epoch 37: accuracy did not improve from 0.81250
733/804 [==========================>...] - ETA: 0s - loss: 0.5380 - accuracy: 0.7359
Epoch 37: accuracy did not improve from 0.81250

Epoch 37: accuracy did not improve from 0.81250

Epoch 37: accuracy did not improve from 0.81250

Epoch 37: accuracy did not improve from 0.81250

Epoch 37: accuracy did not improve from 0.81250

Epoch 37: accuracy did not improve from 0.81250

Epoch 37: accuracy did not improve from 0.81250

Epoch 37: accuracy did not improve from 0.81250
773/804 [===========================>..] - ETA: 0s - loss: 0.5381 - accuracy: 0.7359
Epoch 37: accuracy did not improve from 0.81250

Epoch 37: accuracy did not improve from 0.81250

Epoch 37: accuracy did not improve from 0.81250


Epoch 38: accuracy did not improve from 0.81250

Epoch 38: accuracy did not improve from 0.81250

Epoch 38: accuracy did not improve from 0.81250

Epoch 38: accuracy did not improve from 0.81250

Epoch 38: accuracy did not improve from 0.81250
605/804 [=====================>........] - ETA: 0s - loss: 0.5353 - accuracy: 0.7389
Epoch 38: accuracy did not improve from 0.81250

Epoch 38: accuracy did not improve from 0.81250

Epoch 38: accuracy did not improve from 0.81250

Epoch 38: accuracy did not improve from 0.81250

Epoch 38: accuracy did not improve from 0.81250

Epoch 38: accuracy did not improve from 0.81250

Epoch 38: accuracy did not improve from 0.81250

Epoch 38: accuracy did not improve from 0.81250

Epoch 38: accuracy did not improve from 0.81250
647/804 [=======================>......] - ETA: 0s - loss: 0.5362 - accuracy: 0.7385
Epoch 38: accuracy did not improve from 0.81250

Epoch 38: accuracy did not improve from 0.81250

Epoch 38: accuracy did not improve from 0.81250

450/804 [===============>..............] - ETA: 0s - loss: 0.5365 - accuracy: 0.7390
Epoch 39: accuracy did not improve from 0.81250

Epoch 39: accuracy did not improve from 0.81250

Epoch 39: accuracy did not improve from 0.81250

Epoch 39: accuracy did not improve from 0.81250

Epoch 39: accuracy did not improve from 0.81250

Epoch 39: accuracy did not improve from 0.81250

Epoch 39: accuracy did not improve from 0.81250

Epoch 39: accuracy did not improve from 0.81250
489/804 [=================>............] - ETA: 0s - loss: 0.5369 - accuracy: 0.7380
Epoch 39: accuracy did not improve from 0.81250

Epoch 39: accuracy did not improve from 0.81250

Epoch 39: accuracy did not improve from 0.81250

Epoch 39: accuracy did not improve from 0.81250

Epoch 39: accuracy did not improve from 0.81250

Epoch 39: accuracy did not improve from 0.81250

Epoch 39: accuracy did not improve from 0.81250

Epoch 39: accuracy did not improve from 0.81250
529/804 [==================>...........] - ETA: 

319/804 [==========>...................] - ETA: 0s - loss: 0.5403 - accuracy: 0.7374
Epoch 40: accuracy did not improve from 0.81250

Epoch 40: accuracy did not improve from 0.81250

Epoch 40: accuracy did not improve from 0.81250

Epoch 40: accuracy did not improve from 0.81250

Epoch 40: accuracy did not improve from 0.81250

Epoch 40: accuracy did not improve from 0.81250

Epoch 40: accuracy did not improve from 0.81250

Epoch 40: accuracy did not improve from 0.81250
359/804 [============>.................] - ETA: 0s - loss: 0.5377 - accuracy: 0.7382
Epoch 40: accuracy did not improve from 0.81250

Epoch 40: accuracy did not improve from 0.81250

Epoch 40: accuracy did not improve from 0.81250

Epoch 40: accuracy did not improve from 0.81250

Epoch 40: accuracy did not improve from 0.81250

Epoch 40: accuracy did not improve from 0.81250

Epoch 40: accuracy did not improve from 0.81250

Epoch 40: accuracy did not improve from 0.81250
402/804 [==============>...............] - ETA: 

In [122]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5515 - accuracy: 0.7271 - 254ms/epoch - 946us/step
Loss: 0.5515244603157043, Accuracy: 0.7271137237548828


In [123]:
# Export our model to HDF5 file
nn.save('optimized_5.h5')